### Word Group In Youtube

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "French"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 20000  # native word end index 28,200,1000,5000,10000, 20000, 40000

# word ratio
if word_end == 28 or word_end == 200:
    unique_word_ratio = 70
    include_wordgroup_ratio = 70
elif word_end == 1000 or word_end == 5000:
    unique_word_ratio = 80
    include_wordgroup_ratio = 80
elif word_end == 10000:
    unique_word_ratio = 85
    include_wordgroup_ratio = 85
else:
    unique_word_ratio = 90
    include_wordgroup_ratio = 90

# word usage
if word_end == 28 or word_end == 200 or word_end == 1000 or word_end == 5000 or word_end == 10000:
    word_usage_min = 10
    word_usage_max = 100
elif word_end == 20000:
    word_usage_min = 7
    word_usage_max = 100
else:
    word_usage_min = 5
    word_usage_max = 100

# use for manuel set
#unique_word_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
#include_wordgroup_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
#word_usage_min = 10  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
#word_usage_max = 100


# sentence ratio and time shift
adjust_sent_word_ratio = 15  #28,200 = 15 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
#adjust_text_word_ratio = 5  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"word_end:{word_end}\nfile_ext:{file_ext}\nunique_word_ratio:{unique_word_ratio}\n\
include_wordgroup_ratio:{include_wordgroup_ratio}\nword_usage_min:{word_usage_min}\nword_usage_max:{word_usage_max}")

word_end:40000
file_ext:6
unique_word_ratio:90
include_wordgroup_ratio:90
word_usage_min:5
word_usage_max:100


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Data/1-Word Group In Youtube Sentence"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [6]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [7]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [8]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [9]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [10]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,de,16588988
1,je,16386475
2,pas,11547876
3,le,10592792
4,la,9939090
...,...,...
317922,puisetil,5
317923,shouzheng,5
317924,steinam,5
317925,cochezle,5


In [11]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [12]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,de,16588988
1,je,16386475
2,pas,11547876
3,le,10592792
4,la,9939090
...,...,...
39995,mappellera,342
39996,jinterroge,342
39997,matait,342
39998,titille,342


In [13]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [14]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [15]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

39987

#### Youtube Sentence Analysis

In [16]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.000,00:00:00.200,perrosguirec bretagne,BU7DYGMSUA4
1,00:00:30.972,00:00:33.609,located in the regional natural park of armorica,4nmWfnmVgo4
2,00:00:34.263,00:00:36.866,the huelgoat forest,4nmWfnmVgo4
3,00:00:37.315,00:00:39.315,has always inspired popular stories,4nmWfnmVgo4
4,00:00:40.132,00:00:43.640,places with mysterious names sometimes refer t...,4nmWfnmVgo4
...,...,...,...,...
3010839,00:04:49.620,00:04:54.420,a noter quen cas de décès du titulaire du plan...,Wq8plZLZdzE
3010840,00:04:54.420,00:04:57.540,il ne fait donc pas lobjet dune transmission a...,Wq8plZLZdzE
3010841,00:04:57.540,00:05:01.800,vous souhaitez en savoir plus sur le traitemen...,Wq8plZLZdzE
3010842,00:05:01.800,00:05:05.160,dans ce cas nhésitez pas visiter notre site fi...,Wq8plZLZdzE


In [17]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [18]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,0.000,0.200,perrosguirec bretagne,BU7DYGMSUA4
1,1,30.972,33.609,located in the regional natural park of armorica,4nmWfnmVgo4
2,2,34.263,36.866,the huelgoat forest,4nmWfnmVgo4
3,3,37.315,39.315,has always inspired popular stories,4nmWfnmVgo4
4,4,40.132,43.640,places with mysterious names sometimes refer t...,4nmWfnmVgo4
...,...,...,...,...,...
3010839,3010839,289.620,294.420,a noter quen cas de décès du titulaire du plan...,Wq8plZLZdzE
3010840,3010840,294.420,297.540,il ne fait donc pas lobjet dune transmission a...,Wq8plZLZdzE
3010841,3010841,297.540,301.800,vous souhaitez en savoir plus sur le traitemen...,Wq8plZLZdzE
3010842,3010842,301.800,305.160,dans ce cas nhésitez pas visiter notre site fi...,Wq8plZLZdzE


In [19]:
#df_youtube_sentence[df_youtube_sentence["index"] == 1287503]

In [20]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/(len(sent_word)+0.001))*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [21]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,0.200,perrosguirec bretagne,BU7DYGMSUA4,50.0,perrosguirec,bretagne
1,1,30.972,33.609,located in the regional natural park of armorica,4nmWfnmVgo4,50.0,"natural, regional, armorica, located","the, park, of, in"
2,2,34.263,36.866,the huelgoat forest,4nmWfnmVgo4,66.6,huelgoat,"the, forest"
3,3,37.315,39.315,has always inspired popular stories,4nmWfnmVgo4,40.0,"inspired, stories, popular","has, always"
4,4,40.132,43.640,places with mysterious names sometimes refer t...,4nmWfnmVgo4,66.7,"mysterious, refer, names","arthur, with, places, to, sometimes, king"
...,...,...,...,...,...,...,...,...
3010839,3010839,289.620,294.420,a noter quen cas de décès du titulaire du plan...,Wq8plZLZdzE,86.7,clôturé,"plan, pea, de, a, noter, cas, le, automatiquem..."
3010840,3010840,294.420,297.540,il ne fait donc pas lobjet dune transmission a...,Wq8plZLZdzE,100.0,,"dune, donc, héritiers, fait, pas, aux, il, ne,..."
3010841,3010841,297.540,301.800,vous souhaitez en savoir plus sur le traitemen...,Wq8plZLZdzE,92.3,,"pea, traitement, en, fiscal, france, le, plus,..."
3010842,3010842,301.800,305.160,dans ce cas nhésitez pas visiter notre site fi...,Wq8plZLZdzE,88.9,fiscaloofr,"visiter, dans, cas, notre, ce, site, pas, nhés..."


In [22]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,0.200,perrosguirec bretagne,BU7DYGMSUA4,50.0,perrosguirec,bretagne
1,1,30.972,33.609,located in the regional natural park of armorica,4nmWfnmVgo4,50.0,"natural, regional, armorica, located","the, park, of, in"
2,2,34.263,36.866,the huelgoat forest,4nmWfnmVgo4,66.6,huelgoat,"the, forest"
3,3,37.315,39.315,has always inspired popular stories,4nmWfnmVgo4,40.0,"inspired, stories, popular","has, always"
4,4,40.132,43.640,places with mysterious names sometimes refer t...,4nmWfnmVgo4,66.7,"mysterious, refer, names","arthur, with, places, to, sometimes, king"
...,...,...,...,...,...,...,...,...
3010839,3010839,289.620,294.420,a noter quen cas de décès du titulaire du plan...,Wq8plZLZdzE,86.7,clôturé,"plan, pea, de, a, noter, cas, le, automatiquem..."
3010840,3010840,294.420,297.540,il ne fait donc pas lobjet dune transmission a...,Wq8plZLZdzE,100.0,NaN,"dune, donc, héritiers, fait, pas, aux, il, ne,..."
3010841,3010841,297.540,301.800,vous souhaitez en savoir plus sur le traitemen...,Wq8plZLZdzE,92.3,NaN,"pea, traitement, en, fiscal, france, le, plus,..."
3010842,3010842,301.800,305.160,dans ce cas nhésitez pas visiter notre site fi...,Wq8plZLZdzE,88.9,fiscaloofr,"visiter, dans, cas, notre, ce, site, pas, nhés..."


In [23]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,0.200,perrosguirec bretagne,BU7DYGMSUA4,50.0,perrosguirec,bretagne
1,1,30.972,33.609,located in the regional natural park of armorica,4nmWfnmVgo4,50.0,"natural, regional, armorica, located","the, park, of, in"
2,2,34.263,36.866,the huelgoat forest,4nmWfnmVgo4,66.6,huelgoat,"the, forest"
3,3,37.315,39.315,has always inspired popular stories,4nmWfnmVgo4,40.0,"inspired, stories, popular","has, always"
4,4,40.132,43.640,places with mysterious names sometimes refer t...,4nmWfnmVgo4,66.7,"mysterious, refer, names","arthur, with, places, to, sometimes, king"
...,...,...,...,...,...,...,...,...
2983676,3010839,289.620,294.420,a noter quen cas de décès du titulaire du plan...,Wq8plZLZdzE,86.7,clôturé,"plan, pea, de, a, noter, cas, le, automatiquem..."
2983677,3010840,294.420,297.540,il ne fait donc pas lobjet dune transmission a...,Wq8plZLZdzE,100.0,NaN,"dune, donc, héritiers, fait, pas, aux, il, ne,..."
2983678,3010841,297.540,301.800,vous souhaitez en savoir plus sur le traitemen...,Wq8plZLZdzE,92.3,NaN,"pea, traitement, en, fiscal, france, le, plus,..."
2983679,3010842,301.800,305.160,dans ce cas nhésitez pas visiter notre site fi...,Wq8plZLZdzE,88.9,fiscaloofr,"visiter, dans, cas, notre, ce, site, pas, nhés..."


In [24]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

<ipython-input-24-ca15520b3270>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
5,5,44.193,46.193,sometimes to the devil or the virgin,4nmWfnmVgo4,85.7,NaN,"the, virgin, devil, or, to, sometimes",sometimes to the devil or the virgin
7,7,48.740,53.288,the forest of huelgoat is best known for the c...,4nmWfnmVgo4,66.7,"known, curiosity, huelgoat","the, rocks, of, its, forest, for, beauty, is, ...",and beauty of its rocks
17,17,11.400,15.090,parsemant son cours long de moins de,BYd2A4JSdOo,71.4,parsemant,"de, long, cours, moins, son",son cours long de moins de
18,18,15.090,20.610,est un éloge des eaux dormantes une halte migr...,BYd2A4JSdOo,78.6,"hivernale, dormantes, migratoire","et, des, une, aux, éloge, est, un, halte, oise...",est un éloge des eaux
19,19,20.610,26.550,deau herbiers et roselières sont le refuge des...,BYd2A4JSdOo,84.6,"herbiers, roselières","et, en, le, régression, ou, deau, lotte, sont,...",sont le refuge despèces menacées ou en régress...
...,...,...,...,...,...,...,...,...,...
2983676,3010839,289.620,294.420,a noter quen cas de décès du titulaire du plan...,Wq8plZLZdzE,86.7,clôturé,"plan, pea, de, a, noter, cas, le, automatiquem...",a noter quen cas de décès du titulaire du plan...
2983677,3010840,294.420,297.540,il ne fait donc pas lobjet dune transmission a...,Wq8plZLZdzE,100.0,NaN,"dune, donc, héritiers, fait, pas, aux, il, ne,...",il ne fait donc pas lobjet dune transmission a...
2983678,3010841,297.540,301.800,vous souhaitez en savoir plus sur le traitemen...,Wq8plZLZdzE,92.3,NaN,"pea, traitement, en, fiscal, france, le, plus,...",vous souhaitez en savoir plus sur le traitemen...
2983679,3010842,301.800,305.160,dans ce cas nhésitez pas visiter notre site fi...,Wq8plZLZdzE,88.9,fiscaloofr,"visiter, dans, cas, notre, ce, site, pas, nhés...",dans ce cas nhésitez pas visiter notre site


In [25]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,sometimes to the devil or the virgin,44.193000,46.193000,sometimes to the devil or the virgin,4nmWfnmVgo4
1,and beauty of its rocks,51.888615,53.288000,the forest of huelgoat is best known for the c...,4nmWfnmVgo4
2,son cours long de moins de,12.322500,15.090000,parsemant son cours long de moins de,BYd2A4JSdOo
3,est un éloge des eaux,15.090000,16.518706,est un éloge des eaux dormantes une halte migr...,BYd2A4JSdOo
4,sont le refuge despèces menacées ou en régress...,22.542289,26.550000,deau herbiers et roselières sont le refuge des...,BYd2A4JSdOo
...,...,...,...,...,...
2312496,a noter quen cas de décès du titulaire du plan...,289.620000,294.005185,a noter quen cas de décès du titulaire du plan...,Wq8plZLZdzE
2312497,il ne fait donc pas lobjet dune transmission a...,294.420000,297.540000,il ne fait donc pas lobjet dune transmission a...,Wq8plZLZdzE
2312498,vous souhaitez en savoir plus sur le traitemen...,297.540000,301.800000,vous souhaitez en savoir plus sur le traitemen...,Wq8plZLZdzE
2312499,dans ce cas nhésitez pas visiter notre site,301.800000,304.537778,dans ce cas nhésitez pas visiter notre site fi...,Wq8plZLZdzE


In [26]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,sometimes to the devil or the virgin,43.893000,46.493000,sometimes to the devil or the virgin,4nmWfnmVgo4
1,and beauty of its rocks,51.588615,53.588000,the forest of huelgoat is best known for the c...,4nmWfnmVgo4
2,son cours long de moins de,12.022500,15.390000,parsemant son cours long de moins de,BYd2A4JSdOo
3,est un éloge des eaux,14.790000,16.818706,est un éloge des eaux dormantes une halte migr...,BYd2A4JSdOo
4,sont le refuge despèces menacées ou en régress...,22.242289,26.850000,deau herbiers et roselières sont le refuge des...,BYd2A4JSdOo
...,...,...,...,...,...
2312496,a noter quen cas de décès du titulaire du plan...,289.320000,294.305185,a noter quen cas de décès du titulaire du plan...,Wq8plZLZdzE
2312497,il ne fait donc pas lobjet dune transmission a...,294.120000,297.840000,il ne fait donc pas lobjet dune transmission a...,Wq8plZLZdzE
2312498,vous souhaitez en savoir plus sur le traitemen...,297.240000,302.100000,vous souhaitez en savoir plus sur le traitemen...,Wq8plZLZdzE
2312499,dans ce cas nhésitez pas visiter notre site,301.500000,304.837778,dans ce cas nhésitez pas visiter notre site fi...,Wq8plZLZdzE


In [27]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,sometimes to the devil or the virgin,44,46,sometimes to the devil or the virgin,4nmWfnmVgo4
1,and beauty of its rocks,52,54,the forest of huelgoat is best known for the c...,4nmWfnmVgo4
2,son cours long de moins de,12,15,parsemant son cours long de moins de,BYd2A4JSdOo
3,est un éloge des eaux,15,17,est un éloge des eaux dormantes une halte migr...,BYd2A4JSdOo
4,sont le refuge despèces menacées ou en régress...,22,27,deau herbiers et roselières sont le refuge des...,BYd2A4JSdOo
...,...,...,...,...,...
2312496,a noter quen cas de décès du titulaire du plan...,289,294,a noter quen cas de décès du titulaire du plan...,Wq8plZLZdzE
2312497,il ne fait donc pas lobjet dune transmission a...,294,298,il ne fait donc pas lobjet dune transmission a...,Wq8plZLZdzE
2312498,vous souhaitez en savoir plus sur le traitemen...,297,302,vous souhaitez en savoir plus sur le traitemen...,Wq8plZLZdzE
2312499,dans ce cas nhésitez pas visiter notre site,302,305,dans ce cas nhésitez pas visiter notre site fi...,Wq8plZLZdzE


In [28]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,ici vous devrez apprendre les leçons de chez n...,5564,5630,daccord merci beaucoup john laughland owen pat...,tlPvaDllxiI
1,changer ma vie et de réaliser tout mes rêve et...,0,2,moi cest roland et mon ami ousouf jai contacte...,xdK4jgVZ77c
2,si vous voulez laccord de vendredi saint avec ...,4267,4335,si vous voulez laccord de vendredi saint avec ...,tlPvaDllxiI
3,avec nos voisins et cest fondamental ça parce ...,10156,10222,et ce que nous allons faire nous allons aller ...,tlPvaDllxiI
4,je voudrais juste dire quelque chose ici cest ...,3796,3812,je voudrais juste dire quelque chose ici cest ...,tlPvaDllxiI
...,...,...,...,...,...
2312496,on a vu de la,3077,3080,du camembert camembert brille cest haribo on a...,pdftbIHU25Q
2312497,on a eu de la,138,140,un jour au restou on a eu de la tartiflette,8vqkr1H5cb8
2312498,on a on a le,1143,1144,on a on a le trading volume total,wQngLI2zsls
2312499,o dia de s o,226,228,o dia de s o martinho,49y3w60Dv1A


In [29]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,ici vous devrez apprendre les leçons de chez n...,5564,5630,daccord merci beaucoup john laughland owen pat...,tlPvaDllxiI,60
1,changer ma vie et de réaliser tout mes rêve et...,0,2,moi cest roland et mon ami ousouf jai contacte...,xdK4jgVZ77c,71
2,si vous voulez laccord de vendredi saint avec ...,4267,4335,si vous voulez laccord de vendredi saint avec ...,tlPvaDllxiI,67
3,avec nos voisins et cest fondamental ça parce ...,10156,10222,et ce que nous allons faire nous allons aller ...,tlPvaDllxiI,79
4,je voudrais juste dire quelque chose ici cest ...,3796,3812,je voudrais juste dire quelque chose ici cest ...,tlPvaDllxiI,75
...,...,...,...,...,...,...
2312496,on a vu de la,3077,3080,du camembert camembert brille cest haribo on a...,pdftbIHU25Q,100
2312497,on a eu de la,138,140,un jour au restou on a eu de la tartiflette,8vqkr1H5cb8,100
2312498,on a on a le,1143,1144,on a on a le trading volume total,wQngLI2zsls,60
2312499,o dia de s o,226,228,o dia de s o martinho,49y3w60Dv1A,80


In [30]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-30-d6186b820154>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
<ipython-input-30-d6186b820154>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,détruire cette tyrannie cette prison des peupl...,7753,7777,donc il y a une forte résilience nous contribu...,tlPvaDllxiI
1,ils ne respectent pas toujours la réglementati...,353,371,ils ne respectent pas toujours la réglementati...,fcdtNH07Ip0
2,alors quelle avait tous les atouts dans son je...,4471,4501,et cette andouille de mme may sest présentée l...,tlPvaDllxiI
3,vue se poursuit marc est très intrigué par le ...,409,430,la garde à vue se poursuit marc est très intri...,vhL2RiTHkzE
4,attendent que la nuit tombe pour sortir leur f...,604,626,il y a sa lionne dun an mohamed et son ami has...,_OfVDjtC93Y
...,...,...,...,...,...
1780325,un et là on a,1154,1155,level un et là on a advance level,A742A02QROI
1780326,que a n no ha,204,205,ganancias resultar n en el investimiento del f...,hFw8S3mSYiw
1780327,a un an et il,20,22,il y a un an et il y avait uniquement létage,Z7fTk6UFVfU
1780328,on a vu de la,3077,3080,du camembert camembert brille cest haribo on a...,pdftbIHU25Q


In [31]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-31-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,détruire cette tyrannie cette prison des peupl...,7753,7777,donc il y a une forte résilience nous contribu...,tlPvaDllxiI,46
1,ils ne respectent pas toujours la réglementati...,353,371,ils ne respectent pas toujours la réglementati...,fcdtNH07Ip0,98
2,alors quelle avait tous les atouts dans son je...,4471,4501,et cette andouille de mme may sest présentée l...,tlPvaDllxiI,62
3,vue se poursuit marc est très intrigué par le ...,409,430,la garde à vue se poursuit marc est très intri...,vhL2RiTHkzE,90
4,attendent que la nuit tombe pour sortir leur f...,604,626,il y a sa lionne dun an mohamed et son ami has...,_OfVDjtC93Y,62
...,...,...,...,...,...,...
1780325,un et là on a,1154,1155,level un et là on a advance level,A742A02QROI,39
1780326,que a n no ha,204,205,ganancias resultar n en el investimiento del f...,hFw8S3mSYiw,14
1780327,a un an et il,20,22,il y a un an et il y avait uniquement létage,Z7fTk6UFVfU,29
1780328,on a vu de la,3077,3080,du camembert camembert brille cest haribo on a...,pdftbIHU25Q,15


In [32]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

<ipython-input-32-ea709f90da2b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,ils ne respectent pas toujours la réglementati...,353,371,ils ne respectent pas toujours la réglementati...,fcdtNH07Ip0
1,arriver mathieu ne néglige rien il vise lexcel...,642,662,et pour y arriver mathieu ne néglige rien il v...,OGkVgSZI8f8
2,mais comme toujours ce matériau nest pas parfa...,365,377,mais comme toujours ce matériau nest pas parfa...,e2ITAibdtWo
3,jai mis dautres choses en place de lhypnose de...,52,62,jai mis dautres choses en place de lhypnose de...,2T4jG33szys
4,tout le long du projet il a participé en poste...,290,303,tout le long du projet il a participé en poste...,9g1rjIjSUW0
...,...,...,...,...,...
977734,il ny en a pas,176,177,il ny en a pas,Q0t98lEi0Gs
977735,on a fini en m,41,48,on a fini en m,ymYVMpSDFE0
977736,il ny en a pas,383,388,il ny en a pas,AH7WDk2ue_w
977737,il ny a pas de,670,672,il ny a pas de,FBrRdU_ArfA


In [33]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,lassemblée nationale monsieur lancien président,zzV2t7s8LnU
1,leurope était faite unie contre langleterre,zzV2t7s8LnU
2,lhomme arrêté na rien déclaré,zyN0TmZJIBs
3,elles vont réfléchir différemment dun garçon,zyCA2pqSQ5E
4,après avoir éduqué leurs enfants souvent beauc...,zyCA2pqSQ5E
...,...,...
4031,jessaie dêtre bien droit impeccable quelques a...,0C2Pu7hRiuM
4032,moi même javais beaucoup travaillé sur ces suj...,08ikcqNz89w
4033,ton regard na jamais changé même quand jai fui...,06pc8hDfXiI
4034,selon katherine vaughan williams lartiste,04pwjHyEp3Q


In [34]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,ils ne respectent pas toujours la réglementati...,353,371,ils ne respectent pas toujours la réglementati...,fcdtNH07Ip0
1,arriver mathieu ne néglige rien il vise lexcel...,642,662,et pour y arriver mathieu ne néglige rien il v...,OGkVgSZI8f8
2,mais comme toujours ce matériau nest pas parfa...,365,377,mais comme toujours ce matériau nest pas parfa...,e2ITAibdtWo
3,jai mis dautres choses en place de lhypnose de...,52,62,jai mis dautres choses en place de lhypnose de...,2T4jG33szys
4,tout le long du projet il a participé en poste...,290,303,tout le long du projet il a participé en poste...,9g1rjIjSUW0
...,...,...,...,...,...
3984,ah ouais ok bah oui,779,781,ah ouais ok bah oui,MlvBEH7aY0k
3985,how did he do that,144,145,how did he do that,MCdX2pscGc8
3986,this is two in one,684,685,this is two in one,3_1Scvw8MnA
3987,no but not at all,356,358,no but not at all,GpWqsNJNOVM


In [35]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,ils ne respectent pas toujours la réglementati...,353,371,ils ne respectent pas toujours la réglementati...,fcdtNH07Ip0,https://www.youtube.com/watch?v=fcdtNH07Ip0&t=...
1,arriver mathieu ne néglige rien il vise lexcel...,642,662,et pour y arriver mathieu ne néglige rien il v...,OGkVgSZI8f8,https://www.youtube.com/watch?v=OGkVgSZI8f8&t=...
2,mais comme toujours ce matériau nest pas parfa...,365,377,mais comme toujours ce matériau nest pas parfa...,e2ITAibdtWo,https://www.youtube.com/watch?v=e2ITAibdtWo&t=...
3,jai mis dautres choses en place de lhypnose de...,52,62,jai mis dautres choses en place de lhypnose de...,2T4jG33szys,https://www.youtube.com/watch?v=2T4jG33szys&t=52s
4,tout le long du projet il a participé en poste...,290,303,tout le long du projet il a participé en poste...,9g1rjIjSUW0,https://www.youtube.com/watch?v=9g1rjIjSUW0&t=...
...,...,...,...,...,...,...
3984,ah ouais ok bah oui,779,781,ah ouais ok bah oui,MlvBEH7aY0k,https://www.youtube.com/watch?v=MlvBEH7aY0k&t=...
3985,how did he do that,144,145,how did he do that,MCdX2pscGc8,https://www.youtube.com/watch?v=MCdX2pscGc8&t=...
3986,this is two in one,684,685,this is two in one,3_1Scvw8MnA,https://www.youtube.com/watch?v=3_1Scvw8MnA&t=...
3987,no but not at all,356,358,no but not at all,GpWqsNJNOVM,https://www.youtube.com/watch?v=GpWqsNJNOVM&t=...


In [36]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [37]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [38]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [39]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

['French_Word_Group_Max_In_Youtube_Sentence_40000_Word6.csv',
 'French_Word_Group_Max_In_Youtube_Sentence_40000_Word6.xlsx']

In [40]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [41]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass